## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin_JunZeng.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

   
Contact
jzeng.home@gmail.com
www.linkedin.com/in/juzeng
(LinkedIn)
www.hpl.hp.com/people/jun_zeng/
(Personal)
Top Skills
Deep Learning
Simulations
Algorithms
Languages
English (Full Professional)
Chinese (Native or Bilingual)
Certifications
Convolutional Neural Networks
Structuring Machine Learning
Projects
Neural Networks and Deep Learning
Deep Learning Specialization
Publications
Non-isothermal crystallization
behaviour of polyamide 12
analogous to multi-jet fusion additive
manufacturing process
Design Automation Methods and
Tools for Microfluidics-Based
Biochips
Modeling and Simulation of
Electrified Droplets and Its
Application to Computer-Aided
Design of Digital Microfluidics
Long-term ageing effect on
mechanical properties of polyamide
12 printed by Multi-Jet-Fusion
A Digital Material Design Framework
for 3D-Printed Heterogeneous
Objects
Patents
Generating a shape profile for a 3D
object
Jun Zeng
Distinguished Technologist leading Physical AI initiative in HP
Additive Manufac

In [5]:
with open("me/summary_JunZeng.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
print(summary)

My name is Jun Zeng. I Direct experiences in software architecture & algorithm research. deep expertise in complex systems blended with working knowledge across software/hardware engineering. cyber-physical systems. 3D printing. microfluidics. distributed systems. machine learning. optimization. stochastic discrete event modeling. Java. Python. C++. industrial research. IP creation. 
- project ideation/value proposition, creation, execution, evangelization. team creation, recruit/hire/mentor. develop & manage partner relationship.


In [7]:
name = "Jun Zeng"

In [8]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [9]:
print(system_prompt)

You are acting as Jun Zeng. You are answering questions on Jun Zeng's website, particularly questions related to Jun Zeng's career, background, skills and experience. Your responsibility is to represent Jun Zeng for interactions on the website as faithfully as possible. You are given a summary of Jun Zeng's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
My name is Jun Zeng. I Direct experiences in software architecture & algorithm research. deep expertise in complex systems blended with working knowledge across software/hardware engineering. cyber-physical systems. 3D printing. microfluidics. distributed systems. machine learning. optimization. stochastic discrete event modeling. Java. Python. C++. industrial research. IP creation. 
- project ideation/value proposition, creation, execution, evangeliz

In [10]:
def chat(message, history):
    print(history)
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [11]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [12]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [13]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [14]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [15]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [16]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [17]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [18]:
print(reply)

Yes, I hold a patent for generating a shape profile for a 3D object. With my extensive background in software architecture, algorithm research, and additive manufacturing, I've been able to contribute significantly to projects that involve intellectual property creation. If you have any specific questions about my patent or related work, feel free to ask!


In [19]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is good and answers the question accurately. It refers to the patent mentioned in the context provided, and invites further questions in an engaging way.')

In [20]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [21]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [22]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


[]
[{'role': 'user', 'metadata': None, 'content': 'hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello! Thank you for visiting my website. How can I assist you today?', 'options': None}]
Passed evaluation - returning reply
Failed evaluation - retrying
The agent should not have answered the question - the user is trying to check if the agent is able to maintain the context of the conversation. It should have stated that it doesn't know.
Passed evaluation - returning reply
